In [1]:
import prototypes.BoseHubbardModel as bh
import numpy as np
import pandas as pd
import scipy.optimize as opt
import plotly
import plotly.graph_objects as go
import time

We define the chemical potential in this case as the energy required to add or remove a particle.
\begin{align}
    \mu^p(L) &= E_0(L, N+1) - E_0(L, N) \\
    \mu^h(L) &= E_0(L, N) - E_0(L, N-1)
\end{align}

In [2]:
def chemicalPotential(sites, bosons, boson_limit, t, U, site_potentials, boundary_conditions):
    H_n = bh.BoseHubbardHamiltonian(sites, bosons, t, U, [site_potentials]*sites, boundary_conditions)
    basis_n = bh.ONVBasis(sites, bosons, boson_limit)
    H_n_eval = basis_n.evaluateHamiltonian(H_n)

    H_n_plus_1 = bh.BoseHubbardHamiltonian(sites, bosons+1, t, U, [site_potentials]*sites, boundary_conditions)
    basis_n_plus_1 = bh.ONVBasis(sites, bosons+1, boson_limit)
    H_n_plus_1_eval = basis_n_plus_1.evaluateHamiltonian(H_n_plus_1)

    H_n_min_1 = bh.BoseHubbardHamiltonian(sites, bosons-1, t, U, [site_potentials]*sites, boundary_conditions)
    basis_n_min_1 = bh.ONVBasis(sites, bosons-1, boson_limit)
    H_n_min_1_eval = basis_n_min_1.evaluateHamiltonian(H_n_min_1)

    E_n = bh.BoseHubbardSolver(H_n_eval).groundState().energy
    E_n_plus_1 = bh.BoseHubbardSolver(H_n_plus_1_eval).groundState().energy
    E_n_min_1 = bh.BoseHubbardSolver(H_n_min_1_eval).groundState().energy

    return E_n_plus_1 - E_n, E_n - E_n_min_1

def chemicalPotential_H_eval(H_n_eval, H_n_plus_1_eval, H_n_min_1_eval):

    E_n = bh.BoseHubbardSolver(H_n_eval, True).groundState().energy
    E_n_plus_1 = bh.BoseHubbardSolver(H_n_plus_1_eval, True).groundState().energy
    E_n_min_1 = bh.BoseHubbardSolver(H_n_min_1_eval, True).groundState().energy

    return E_n_plus_1 - E_n, E_n - E_n_min_1

In [3]:
def generate_H(sites, bosons, boson_limit, U, site_potentials, boundary_conditions):
    H_diag = bh.BoseHubbardHamiltonian(sites, bosons, 0, U, [site_potentials]*sites, boundary_conditions)
    H_hop = bh.BoseHubbardHamiltonian(sites, bosons, 1, 0, [0]*sites, boundary_conditions)
    basis = bh.ONVBasis(sites, bosons, boson_limit)
    H_diag_eval = basis.evaluateHamiltonian(H_diag)
    H_hop_eval = basis.evaluateHamiltonian(H_hop)

    return H_diag_eval, H_hop_eval

In [33]:
def mottLobe(sites, bosons):
     H_n_eval_diag, H_n_eval_hop = generate_H(sites, bosons, None, 2, 0, True)
     H_n_plus_1_eval_diag, H_n_plus_1_eval_hop = generate_H(sites, bosons + 1, None, 2, 0, True)
     H_n_min_1_eval_diag, H_n_min_1_eval_hop = generate_H(sites, bosons - 1, None, 2, 0, True)

     chemical_potentials_upper = []
     chemical_potentials_lower = []
     t_U_values = []

     for t in np.arange(0.0, 0.801, 0.05):
          H_n_eval = H_n_eval_diag + H_n_eval_hop * t
          H_n_plus_1_eval = H_n_plus_1_eval_diag + H_n_plus_1_eval_hop * t
          H_n_min_1_eval = H_n_min_1_eval_diag + H_n_min_1_eval_hop * t
          mu_upper, mu_lower = chemicalPotential_H_eval(H_n_eval, H_n_plus_1_eval, H_n_min_1_eval)
          chemical_potentials_upper.append(mu_upper/2) # divided by U
          chemical_potentials_lower.append(mu_lower/2) # divided by U
          t_U_values.append(t/2) # divide by U

     return [chemical_potentials_upper, chemical_potentials_lower, t_U_values]

In [34]:
ML1 = mottLobe(6, 6)

In [35]:
ML2 = mottLobe(6, 12)

In [36]:
ML3 = mottLobe(6, 18)

In [37]:
def plot(*lobes):

    figure = go.Figure()
    colors = plotly.colors.DEFAULT_PLOTLY_COLORS
    
    for i, lobe in enumerate(lobes):
        figure.add_trace(go.Scatter(
            name= 'ρ<sub>site</sub>=' + str(i+1),
            mode = 'markers+lines',
            marker=dict(color=colors[i].replace(")", "").replace("rgb", "rgba")+', 1)', size=5),
            line=dict(color=colors[i].replace(")", "").replace("rgb", "rgba")+', 0.3)', width=5),
            x = lobe[2],
            y = lobe[0]))

        figure.add_trace(go.Scatter(
            mode = 'markers+lines',
            marker=dict(color=colors[i].replace(")", "").replace("rgb", "rgba")+', 1)', size=5),
            line=dict(color=colors[i].replace(")", "").replace("rgb", "rgba")+', 0.3)', width=5),
            fill="tonexty",
            fillcolor=colors[i].replace(")", "").replace("rgb", "rgba")+', 0.3)',
            x = lobe[2],
            y = lobe[1],
            showlegend=False))
    

    figure.update_xaxes(title_text='t/U (a.u.)',
                        ticks="inside",
                        nticks = 5, showgrid=True, zeroline=True, linecolor="black", gridcolor="gray", zerolinecolor="black", tickfont=dict(size=24), tickformat=".1f", range=[0, 0.4])
    figure.update_yaxes(title_text='μ/U (a.u.)',
                        ticks="inside", showgrid=True, zeroline=True, linecolor="black", gridcolor="gray", zerolinecolor="black", tickfont=dict(size=24), tickformat=".2f")
    figure.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    figure.update_layout(font=dict(size=26))
    figure.update_layout(height = 800, width = 1200)

    figure.show()

    figure.write_image("./figures/Mott-Lobes-ED.pdf")
    time.sleep(5)
    figure.write_image("./figures/Mott-Lobes-ED.pdf")

In [38]:
plot(ML1, ML2, ML3)